In [108]:
import pytz
from datetime import datetime, date, timedelta
import pandas as pd
import altair as alt
import altair_latimes as lat

In [109]:
alt.themes.register('latimes', lat.theme)
alt.themes.enable('latimes')

ThemeRegistry.enable('latimes')

In [110]:
pd.set_option('display.float_format', lambda x: '%.3f' % x)

### Import

In [111]:
df = pd.read_csv(
    "../data/raw/reservoirs/major-reservoir-scrape-latest.csv", 
    parse_dates=["DATE TIME", "OBS DATE"]
)

In [112]:
hist_df = pd.read_csv(
    "../data/metadata/reservoirs-historical-averages.csv"
)

In [113]:
details_df = pd.read_csv(
    "../data/metadata/reservoirs-metadata-details.csv"
)

### Clean

In [114]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6496 entries, 0 to 6495
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   STATION_ID     6496 non-null   object        
 1   DURATION       6496 non-null   object        
 2   SENSOR_NUMBER  6496 non-null   int64         
 3   SENSOR_TYPE    6496 non-null   object        
 4   DATE TIME      6496 non-null   datetime64[ns]
 5   OBS DATE       6496 non-null   datetime64[ns]
 6   VALUE          6496 non-null   object        
 7   DATA_FLAG      6496 non-null   object        
 8   UNITS          6496 non-null   object        
dtypes: datetime64[ns](2), int64(1), object(6)
memory usage: 456.9+ KB


In [115]:
df.columns = df.columns.str.lower()

In [116]:
df.columns = df.columns.str.replace(" ","_")

In [117]:
df = df.rename(columns={
    "station_id": "reservoir_id",
    "date_time": "date",
    "value": "storage_af"
})

In [118]:
df["storage_af"] = df["storage_af"].str.replace("---","")

In [119]:
df["storage_af"] = pd.to_numeric(df["storage_af"])

### Merge historical average

Create month column for merging

In [120]:
df["month"] = pd.DatetimeIndex(df.date).month

In [121]:
hist_df["month"] = pd.to_datetime(hist_df["month"], format='%B').dt.month

In [122]:
merge_historical = pd.merge(
    df[["reservoir_id","date","month","storage_af","units"]],
    hist_df[["reservoir_id", "month", "average_storage_value", "average_storage_unit"]],
    how="left",
    on=["reservoir_id","month"]
)

In [123]:
details_df.columns

Index(['id', 'name', 'lake', 'stream', 'capacity', 'url', 'elevation', 'basin',
       'county', 'hydrologic_region', 'nearby_city', 'lat', 'lon', 'operator',
       'maintenance'],
      dtype='object')

In [124]:
merge_details = pd.merge(
    merge_historical,
    details_df[["id", "lake", "capacity"]],
    how="left",
    left_on=["reservoir_id"],
    right_on=["id"]
).drop(["id", "units", "average_storage_unit"], axis=1)

### Calculate percentages

In [125]:
merge_details["current_level_pct_of_total"] = merge_details["storage_af"] / merge_details["capacity"]

In [126]:
merge_details["average_level_pct_of_total"] = merge_details["average_storage_value"] / merge_details["capacity"]

In [127]:
merge_details["current_level_pct_of_avg"] = merge_details["storage_af"] / merge_details["average_storage_value"]

### Check out latest data

First drop na rows

In [128]:
drop_na_df = merge_details.dropna(subset=["storage_af"])

In [154]:
tz = pytz.timezone("America/Los_Angeles")

In [155]:
today = datetime.now(tz).date()
today

datetime.date(2022, 6, 20)

In [156]:
yesterday = (today - pd.DateOffset(days=1)).date()
yesterday

datetime.date(2022, 6, 19)

In [163]:
latest_df = drop_na_df[drop_na_df.date == pd.to_datetime(yesterday)].copy()

In [164]:
len(latest_df)

138

In [165]:
latest_df.storage_af.sum()

17894232.0

### Filter to just "major" reservoirs

In [171]:
# src: https://cdec.water.ca.gov/resapp/RescondMain
major_reservoirs_list = [
    'SHA', # Shasta
    'ORO', # Oroville
    'CLE', # Trinity Lake
    'NML', # New Melones Reservoir
    'SNL', # San Luis
    'DNP', # New Don Pedro
    'ALM', # Lake Almanor
    'FOL', # Folsom    
    'BUL',
    'CMN', 
    'NML',
    'DNP', # New Don Pedro Reservoir
    'EXC', # Lake McClure
    'WRS',
    'SNL',
    'CCH',
    'CAS',
    'CSI',
    'DMV',
    'MIL',
    'PNF',
]

In [172]:
major_reservoirs = merge_details[merge_details.reservoir_id.isin(major_reservoirs_list)]

Drop missing values

In [173]:
drop_na = major_reservoirs.dropna(subset=["storage_af","average_storage_value"])

In [174]:
len(drop_na)

521

In [177]:
melt=pd.melt(drop_na, id_vars="reservoir_id", value_vars=["storage_af", "average_storage_value", "capacity"])

In [179]:
bar_order = {
    "storage_af": 1, 
    "average_storage_value": 2, 
    "capacity":3            
}

In [180]:
melt["bar_order"] = melt.variable.map(bar_order)

In [181]:
domain = ["capacity", "average_storage_value", "storage_af", ]
range_ = ['#ddd', '#83c6e0', '#1281aa', ]

alt.Chart(melt).mark_bar().encode(
    x=alt.X('reservoir_id', axis=alt.Axis(labels=False)),
    y=alt.Y('value',stack="normalize"),
    order="bar_order",
    color=alt.Color(
        'variable', 
        scale=alt.Scale(domain=domain, range=range_), 
        #sort='descending'
    ),
    tooltip=["reservoir_id"]
).properties(title="California's largest reservoirs")

alt.Chart(...)

In [182]:
domain = ["capacity", "average_storage_value", "storage_af", ]
range_ = ['#ddd', '#83c6e0', '#1281aa', ]

alt.Chart(melt[melt.variable != "capacity_value"]).mark_bar().encode(
    x=alt.X('reservoir_id', axis=alt.Axis(labels=False)),
    y=alt.Y('value',stack="normalize"),
    order="bar_order",
    color=alt.Color(
        'variable', 
        scale=alt.Scale(domain=domain, range=range_), 
        #sort='descending'
    ),
    tooltip=["reservoir_id"]
).properties(title="California's largest reservoirs")

alt.Chart(...)

In [183]:
domain = ["capacity", "average_storage_value", "storage_af", ]
range_ = ['#ddd', '#83c6e0', '#1281aa', ]

alt.Chart(melt[melt.reservoir_id=="ORO"]).mark_bar().encode(
    x=alt.X('reservoir_id', axis=alt.Axis(labels=False)),
    y=alt.Y('value',stack="normalize"),
    order="bar_order",
    color=alt.Color(
        'variable', 
        scale=alt.Scale(domain=domain, range=range_), 
        #sort='descending'
    ),
    tooltip=["reservoir_id"]
).properties(title="Lake Oroville", width=150)

alt.Chart(...)

### Export

Trim and rename for export

In [187]:
export_df = drop_na[
    ['reservoir_id','lake', 'date', 'storage_af', 'average_storage_value', 'capacity', 'current_level_pct_of_total',
       'average_level_pct_of_total', 'current_level_pct_of_avg' ]
].rename(columns={
    'lake': 'reservoir_name',
    'average_storage_value': 'historical_average',
    'capacity': 'total_capacity'
})

In [188]:
export_df.to_csv("../data/processed/reservoirs/reservoirs-latest.csv", index=False)

In [189]:
export_df.to_csv(f"../data/processed/reservoirs/reservoirs-{today}.csv", index=False)

---

### Merge geo data

In [ ]:
metadata_df = pd.read_csv("../data/metadata/reservoirs-metadata-details.csv")

In [ ]:
metadata_df["lat"] = metadata_df["lat"].str.replace("°","")
metadata_df["lon"] = metadata_df["lon"].str.replace("°","")

In [ ]:
merge_coords = pd.merge(
    drop_na,
    metadata_df[["id","lat","lon"]],
    how="left",
    left_on=["reservoir_id"],
    right_on=["id"]
).drop(["id","month","units","average_storage_unit","capacity_unit"], axis=1)

In [ ]:
merge_coords.lake_name = merge_coords.lake_name.str.strip()

In [ ]:
len(merge_coords)

In [ ]:
merge_coords#[merge_coords.lake_name=="Lake Shasta"]

In [ ]:
merge_coords.to_csv("../data/processed/reservoirs/reservoirs-with-points.csv", index=False)